<a href="https://colab.research.google.com/github/gabrieldilay/rossmann-forecast-mvp/blob/main/rossmann_faturamento_mvp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install -q gdown
import pandas as pd

# IDs dos arquivos no Google Drive
id_train = "1N9hI0gq31AxS_1N8_ua9YrV2Jf_X43vn"
id_test  = "1feOAfTaB3sniWHWvfjyQ31iixDk5cO2s"
id_store = "1PT4R3yX-wSNuPXLMWmfr1gSN5BjzyaAh"

# URLs de download direto
url_train = f"https://drive.google.com/uc?id={id_train}"
url_test  = f"https://drive.google.com/uc?id={id_test}"
url_store = f"https://drive.google.com/uc?id={id_store}"

# Baixar arquivos para o Colab
!gdown {url_train} -O train.csv
!gdown {url_test}  -O test.csv
!gdown {url_store} -O store.csv

# Ler os datasets
train = pd.read_csv("train.csv", parse_dates=["Date"])
test  = pd.read_csv("test.csv",  parse_dates=["Date"])
store = pd.read_csv("store.csv")

# Conferir primeiras linhas
print("Train.csv")
display(train.head())

print("\nTest.csv")
display(test.head())

print("\nStore.csv")
display(store.head())

Downloading...
From: https://drive.google.com/uc?id=1N9hI0gq31AxS_1N8_ua9YrV2Jf_X43vn
To: /content/train.csv
100% 38.1M/38.1M [00:00<00:00, 255MB/s]
Downloading...
From: https://drive.google.com/uc?id=1feOAfTaB3sniWHWvfjyQ31iixDk5cO2s
To: /content/test.csv
100% 1.43M/1.43M [00:00<00:00, 103MB/s]
Downloading...
From: https://drive.google.com/uc?id=1PT4R3yX-wSNuPXLMWmfr1gSN5BjzyaAh
To: /content/store.csv
100% 45.0k/45.0k [00:00<00:00, 59.6MB/s]
Train.csv


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1



Test.csv


,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0
3,4,8,4,2015-09-17,1.0,1,0,0
4,5,9,4,2015-09-17,1.0,1,0,0



Store.csv


,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [7]:
# ============================================================
# PASSO 2 — EDA rápido + checagens + merge com 'store'
# Explico o porquê de cada passo nos comentários.
# ============================================================

import numpy as np
import pandas as pd

# 1) Visão geral básica: tamanho, colunas e tipos
print("SHAPES:", train.shape, test.shape, store.shape)
print("\n== TRAIN info ==")
train.info()
print("\n== TEST info ==")
test.info()
print("\n== STORE info ==")
store.info()

# 2) Período de datas na base de treino/teste (série temporal!)
def period_report(df, name):
    # justificativa: entender cobertura temporal e consistência entre splits
    print(f"\n{name} período:", df['Date'].min().date(), "→", df['Date'].max().date())
    # lojas e observações
    print(f"{name} lojas únicas:", df['Store'].nunique())

period_report(train, "TRAIN")
period_report(test,  "TEST")

# 3) Checagens de chave e duplicidade (cada linha é um dia por loja?)
# justificativa: garantir consistência (uma linha por Store+Date no train)
dup_train = train.duplicated(subset=['Store','Date']).sum()
print("\nDuplicatas em TRAIN por (Store, Date):", dup_train)

# 4) Valores únicos de variáveis categóricas principais (insumos de sazonalidade/promo)
def quick_uniques(df, cols, name):
    print(f"\nValores únicos — {name}")
    for c in cols:
        if c in df.columns:
            print(f"{c}: {df[c].dropna().unique()[:10]} (n_unique={df[c].nunique()})")

quick_uniques(train, ['DayOfWeek','Open','Promo','StateHoliday','SchoolHoliday'], "TRAIN")
quick_uniques(test,  ['DayOfWeek','Open','Promo','StateHoliday','SchoolHoliday'], "TEST")
quick_uniques(store, ['StoreType','Assortment','Promo2','PromoInterval'], "STORE")

# 5) Normalização leve de StateHoliday (pode vir '0','a','b','c' como string)
# justificativa: padronizar tipo para evitar bugs na engenharia de atributos
for df in (train, test):
    if df['StateHoliday'].dtype != 'object':
        df['StateHoliday'] = df['StateHoliday'].astype(str)
    df['StateHoliday'] = df['StateHoliday'].replace({'0.0':'0','0':'0'})

quick_uniques(train, ['StateHoliday'], "TRAIN (normalizado)")
quick_uniques(test,  ['StateHoliday'], "TEST  (normalizado)")

# 6) Checagem importante: dias fechados (Open=0) deveriam ter Sales=0
# justificativa: isso evita viés no treino. alguns kernels removem linhas com Open=0.
closed = train.query('Open==0')
inconsist = closed.query('Sales>0').shape[0]
print(f"\nDias com Open=0 no TRAIN: {closed.shape[0]} | com Sales>0 (inconsistência): {inconsist}")

# 7) Ausências (top 15) antes do merge
def na_report(df, name):
    na = df.isna().sum().sort_values(ascending=False)
    na = na[na>0]
    print(f"\nNAs em {name} (top 15):")
    print(na.head(15))

na_report(train, "TRAIN (antes do merge)")
na_report(test,  "TEST  (antes do merge)")
na_report(store, "STORE")

# 8) Merge com 'store' — traz variáveis exógenas (tipo de loja, competição, promo2)
# justificativa: essas features são clássicas para melhorar predição de vendas
train_full = train.merge(store, on='Store', how='left', validate='many_to_one')
test_full  = test.merge(store,  on='Store', how='left', validate='many_to_one')

print("\nShapes pós-merge:", train_full.shape, test_full.shape)
na_report(train_full, "TRAIN_FULL (pós-merge)")
na_report(test_full,  "TEST_FULL  (pós-merge)")

# 9) Checks de sanidade pós-merge
# justificativa: garantir que nenhuma loja do train/test ficou sem seu cadastro (store)
missing_store_train = train_full['StoreType'].isna().sum()
missing_store_test  = test_full['StoreType'].isna().sum()
print(f"\nLinhas sem cadastro de loja (StoreType NA): TRAIN={missing_store_train}, TEST={missing_store_test}")

# 10) Feature de calendário (prévia) — ajuda na agregação mensal depois
# justificativa: faremos previsão mensal/semestral/anual → já extraio componentes de data
for df in (train_full, test_full):
    df['Year']  = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month
    df['Week']  = df['Date'].dt.isocalendar().week.astype(int)
    df['Day']   = df['Date'].dt.day
    df['dow']   = df['Date'].dt.dayofweek  # 0=Mon ... 6=Sun

# 11) Sumarização rápida — como estão as vendas por mês (cadeia toda)
# justificativa: visão de sazonalidade mensal para guiar nosso plano de modelagem
monthly_chain = (
    train_full
    .query("Open==1")                   # evitamos dias fechados
    .groupby(['Year','Month'], as_index=False)['Sales']
    .sum()
    .sort_values(['Year','Month'])
)
print("\nPrévia: faturamento mensal (soma de Sales) — cadeia inteira:")
print(monthly_chain.head(12))
print("...\n", monthly_chain.tail(12))

# 12) Pequena verificação por loja (opcional agora): quantas lojas e distribuição de vendas
store_sales = (
    train_full
    .query("Open==1")
    .groupby('Store', as_index=False)['Sales'].sum()
    .rename(columns={'Sales':'Sales_total'})
)
print("\nLojas (amostra) e vendas totais (para termos noção de heterogeneidade):")
print(store_sales.sample(5, random_state=42))


SHAPES: (1017209, 9) (41088, 8) (1115, 10)

== TRAIN info ==
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 9 columns):
 #   Column         Non-Null Count    Dtype         
---  ------         --------------    -----         
 0   Store          1017209 non-null  int64         
 1   DayOfWeek      1017209 non-null  int64         
 2   Date           1017209 non-null  datetime64[ns]
 3   Sales          1017209 non-null  int64         
 4   Customers      1017209 non-null  int64         
 5   Open           1017209 non-null  int64         
 6   Promo          1017209 non-null  int64         
 7   StateHoliday   1017209 non-null  object        
 8   SchoolHoliday  1017209 non-null  int64         
dtypes: datetime64[ns](1), int64(7), object(1)
memory usage: 69.8+ MB

== TEST info ==
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41088 entries, 0 to 41087
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype        

In [8]:
# Passo 1 - Merge do train com store
df = pd.merge(train, store, on="Store", how="left")

# Passo 2 - Remover linhas onde a loja está fechada (Open = 0) ou vendas zeradas
df = df[(df["Open"] == 1) & (df["Sales"] > 0)]

# Passo 3 - Criar variáveis de tempo
df["Year"] = df["Date"].dt.year
df["Month"] = df["Date"].dt.month
df["Day"] = df["Date"].dt.day
df["DayOfWeek"] = df["Date"].dt.dayofweek
df["WeekOfYear"] = df["Date"].dt.isocalendar().week.astype(int)

# Passo 4 - Conferir resultado
df.head()


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,...,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Year,Month,Day,WeekOfYear
0,1,4,2015-07-31,5263,555,1,1,0,1,c,...,9.0,2008.0,0,NaN,NaN,NaN,2015,7,31,31
1,2,4,2015-07-31,6064,625,1,1,0,1,a,...,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct",2015,7,31,31
2,3,4,2015-07-31,8314,821,1,1,0,1,a,...,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct",2015,7,31,31
3,4,4,2015-07-31,13995,1498,1,1,0,1,c,...,9.0,2009.0,0,NaN,NaN,NaN,2015,7,31,31
4,5,4,2015-07-31,4822,559,1,1,0,1,a,...,4.0,2015.0,0,NaN,NaN,NaN,2015,7,31,31


In [9]:
from sklearn.model_selection import train_test_split

# 1) Selecionar features (X) e target (y)
# Target = 'Sales'
target = "Sales"

# Features numéricas e categóricas (além de variáveis de tempo já criadas)
features = [
    "Store", "Customers", "Promo", "StateHoliday", "SchoolHoliday",
    "StoreType", "Assortment", "CompetitionDistance", "Promo2",
    "Year", "Month", "Day", "DayOfWeek", "WeekOfYear"
]

X = df[features].copy()
y = df[target].copy()

# 2) One-hot encoding para variáveis categóricas
X_encoded = pd.get_dummies(X, columns=["StoreType","Assortment","StateHoliday"], drop_first=True)

print("Shape original:", X.shape)
print("Shape após one-hot:", X_encoded.shape)

# 3) Split treino/validação
# importante: para séries temporais, vamos usar uma divisão temporal simples.
# aqui vamos separar os últimos 3 meses para validação, por exemplo.
cutoff_date = df["Date"].max() - pd.DateOffset(months=3)

X_train = X_encoded[df["Date"] <= cutoff_date]
y_train = y[df["Date"] <= cutoff_date]

X_val = X_encoded[df["Date"] > cutoff_date]
y_val = y[df["Date"] > cutoff_date]

print("Treino:", X_train.shape, "Validação:", X_val.shape)


Shape original: (844338, 14)
Shape após one-hot: (844338, 20)
Treino: (759848, 20) Validação: (84490, 20)


In [11]:
from sklearn.impute import SimpleImputer

# 1) Conferir quais colunas têm NaN
print("Colunas com NaN antes da imputação:")
print(X_encoded.isna().sum()[X_encoded.isna().sum()>0])

# 2) Estratégias de imputação:
# - numéricas: substituímos por mediana (robusto a outliers)
# - categóricas já foram dummificadas, então só sobram numéricas
imputer = SimpleImputer(strategy="median")

X_imputed = pd.DataFrame(imputer.fit_transform(X_encoded), columns=X_encoded.columns, index=X_encoded.index)

# 3) Repetir split temporal
cutoff_date = df["Date"].max() - pd.DateOffset(months=3)

X_train = X_imputed[df["Date"] <= cutoff_date]
y_train = y[df["Date"] <= cutoff_date]

X_val = X_imputed[df["Date"] > cutoff_date]
y_val = y[df["Date"] > cutoff_date]

print("Após imputação:", X_train.shape, X_val.shape)
print("NaNs restantes:", X_train.isna().sum().sum(), X_val.isna().sum().sum())


Colunas com NaN antes da imputação:
CompetitionDistance    2186
dtype: int64
Após imputação: (759848, 20) (84490, 20)
NaNs restantes: 0 0


In [12]:
# ===========================
# Baseline: Regressão Linear
# (agora com dados imputados)
# ===========================

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import pandas as pd

# 1) instancio o modelo mais simples possível
#    justificativa: baseline interpretável e rápido; serve de referência
lr = LinearRegression()

# 2) treino o modelo somente com dados "do passado" (X_train → y_train)
#    justificativa: evitar vazamento; nossa validação é temporal (últimos 3 meses)
lr.fit(X_train, y_train)

# 3) prevejo no período de validação (futuro em relação ao treino)
y_pred = lr.predict(X_val)

# 4) avalio com métricas de regressão que vamos usar ao longo do projeto
mae  = mean_absolute_error(y_val, y_pred)                               # erro médio absoluto (R$)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))                       # raiz do erro quadrático médio
mape = np.mean(np.abs((y_val - y_pred) / y_val)) * 100                  # erro percentual médio (%)
r2   = r2_score(y_val, y_pred)                                          # proporção da variância explicada

print("📊 Avaliação - Regressão Linear (baseline)")
print(f"MAE : {mae:,.2f}")
print(f"RMSE: {rmse:,.2f}")
print(f"MAPE: {mape:.2f}%")
print(f"R²  : {r2:.3f}")

# 5) amostra: reais vs previstos (para checagem rápida)
comparison = pd.DataFrame({
    "Real": y_val.values[:15],
    "Previsto": np.round(y_pred[:15], 2)
})
display(comparison)

# 6) opcional: ver importância/coeficientes (interpretação do baseline)
#    justificativa: entender sinais/impacto linear das features mais diretas
coef_df = pd.DataFrame({
    "feature": X_train.columns,
    "coef": lr.coef_
}).sort_values("coef", ascending=False)

print("\nTop 10 coeficientes positivos (impacto linear ↑ nas vendas):")
display(coef_df.head(10))

print("Top 10 coeficientes negativos (impacto linear ↓ nas vendas):")
display(coef_df.tail(10))


📊 Avaliação - Regressão Linear (baseline)
MAE : 972.08
RMSE: 1,345.20
MAPE: 14.09%
R²  : 0.808


,Real,Previsto
0,5263,5600.89
1,6064,6540.87
2,8314,8329.84
3,13995,12830.38
4,4822,6499.40
5,5651,5967.59
6,15344,12946.79
7,8492,7948.66
8,8565,7030.91
9,7185,6717.54



Top 10 coeficientes positivos (impacto linear ↑ nas vendas):


,feature,coef
19,StateHoliday_c,2745.583623
2,Promo,1137.958147
13,StoreType_d,1126.035424
5,Promo2,300.963380
15,Assortment_c,297.785908
18,StateHoliday_b,261.898076
17,StateHoliday_a,204.176335
6,Year,173.651980
16,StateHoliday_0,170.501610
7,Month,42.803966


Top 10 coeficientes negativos (impacto linear ↓ nas vendas):


,feature,coef
3,SchoolHoliday,40.557398
1,Customers,7.368561
8,Day,1.103659
4,CompetitionDistance,0.025431
0,Store,-0.109178
10,WeekOfYear,-2.143202
9,DayOfWeek,-38.183594
12,StoreType_c,-141.124460
11,StoreType_b,-3250.177627
14,Assortment_b,-4013.307909


In [14]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# 1) instancio o modelo de árvore
tree = DecisionTreeRegressor(max_depth=10, random_state=42)

# 2) treino
tree.fit(X_train, y_train)

# 3) predição
y_pred_tree = tree.predict(X_val)

# 4) métricas
mae_tree  = mean_absolute_error(y_val, y_pred_tree)
rmse_tree = np.sqrt(mean_squared_error(y_val, y_pred_tree))  # cálculo manual
mape_tree = np.mean(np.abs((y_val - y_pred_tree) / y_val)) * 100
r2_tree   = r2_score(y_val, y_pred_tree)

print("📊 Avaliação - Árvore de Regressão")
print(f"MAE : {mae_tree:.2f}")
print(f"RMSE: {rmse_tree:.2f}")
print(f"MAPE: {mape_tree:.2f}%")
print(f"R²  : {r2_tree:.3f}")


📊 Avaliação - Árvore de Regressão
MAE : 822.54
RMSE: 1131.31
MAPE: 11.37%
R²  : 0.864


In [15]:
# ============================================
# Experimento: efeito da profundidade (max_depth)
# Objetivo: ver under/overfitting na prática
# ============================================

import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# grades de profundidade para testar
depth_grid = [3, 5, 7, 9, 12, 15, 20, None]  # None = sem limite (tende a overfit)

results = []

for d in depth_grid:
    # 1) instancio a árvore com profundidade 'd'
    #    (random_state fixo para reprodutibilidade)
    tree = DecisionTreeRegressor(max_depth=d, random_state=42)

    # 2) treino no período de treino
    tree.fit(X_train, y_train)

    # 3) predição na validação (período futuro)
    y_pred = tree.predict(X_val)

    # 4) métricas
    mae  = mean_absolute_error(y_val, y_pred)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    mape = np.mean(np.abs((y_val - y_pred) / y_val)) * 100
    r2   = r2_score(y_val, y_pred)

    # guardo os resultados
    results.append({
        "max_depth": d if d is not None else "None",
        "MAE": mae,
        "RMSE": rmse,
        "MAPE(%)": mape,
        "R2": r2
    })

# 5) organizo em DataFrame e ordeno por RMSE (menor = melhor)
df_depth = pd.DataFrame(results)
df_depth_sorted = df_depth.sort_values("RMSE").reset_index(drop=True)

print("Resultados por profundidade (ordenado por RMSE):")
display(df_depth_sorted)

# 6) também mostro na ordem original para enxergar a tendência com o aumento da profundidade
print("Resultados na ordem testada (tendência under→over):")
display(pd.DataFrame(results))


Resultados por profundidade (ordenado por RMSE):


,max_depth,MAE,RMSE,MAPE(%),R2
0,None,503.336028,729.911636,7.063681,0.943478
1,20,501.980804,737.753706,6.911885,0.942257
2,15,617.726919,885.358027,8.452657,0.916840
3,12,747.044379,1041.712361,10.263468,0.884874
4,9,853.543240,1172.917386,11.809955,0.854047
5,7,932.208541,1294.223653,12.790850,0.822297
6,5,1049.649563,1450.772501,14.536609,0.776707
7,3,1299.348111,1795.728829,18.551664,0.657896


Resultados na ordem testada (tendência under→over):


,max_depth,MAE,RMSE,MAPE(%),R2
0,3,1299.348111,1795.728829,18.551664,0.657896
1,5,1049.649563,1450.772501,14.536609,0.776707
2,7,932.208541,1294.223653,12.790850,0.822297
3,9,853.543240,1172.917386,11.809955,0.854047
4,12,747.044379,1041.712361,10.263468,0.884874
5,15,617.726919,885.358027,8.452657,0.916840
6,20,501.980804,737.753706,6.911885,0.942257
7,None,503.336028,729.911636,7.063681,0.943478


In [16]:
# ============================================
# RANDOM FOREST — baseline + avaliação
# ============================================

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import pandas as pd

# 1) instancio um RF "conservador"
# n_estimators: nº de árvores (mais árvores → mais estável, mais custo)
# max_depth: limite da profundidade (controla complexidade)
# min_samples_leaf: folhas com poucas amostras costumam overfit → subo um pouco
# max_features: nº de features testadas por split; 'sqrt' é um bom default em ensembles
rf = RandomForestRegressor(
    n_estimators=200,
    max_depth=15,            # ponto de partida próximo do que funcionou bem na sua árvore
    min_samples_leaf=2,      # regularização leve
    max_features='sqrt',     # diversifica árvores
    n_jobs=-1,               # usa todos os cores disponíveis
    random_state=42,
    oob_score=False          # deixo False porque estamos avaliando em holdout temporal
)

# 2) treino apenas no período de treino (passado)
rf.fit(X_train, y_train)

# 3) predição no período de validação (futuro)
y_pred_rf = rf.predict(X_val)

# 4) métricas
mae  = mean_absolute_error(y_val, y_pred_rf)
rmse = np.sqrt(mean_squared_error(y_val, y_pred_rf))
mape = np.mean(np.abs((y_val - y_pred_rf) / y_val)) * 100
r2   = r2_score(y_val, y_pred_rf)

print("📊 Avaliação - Random Forest (baseline)")
print(f"MAE : {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"MAPE: {mape:.2f}%")
print(f"R²  : {r2:.3f}")

# 5) amostra real vs previsto
preview = pd.DataFrame({"Real": y_val.values[:15], "Previsto_RF": np.round(y_pred_rf[:15], 2)})
display(preview)

# 6) importância de variáveis (média das reduções de impureza)
imp = pd.DataFrame({
    "feature": X_train.columns,
    "importance": rf.feature_importances_
}).sort_values("importance", ascending=False)

print("\nTop 15 features mais importantes (RF):")
display(imp.head(15))


📊 Avaliação - Random Forest (baseline)
MAE : 775.81
RMSE: 1074.70
MAPE: 11.03%
R²  : 0.877


,Real,Previsto_RF
0,5263,5500.21
1,6064,6392.69
2,8314,8830.89
3,13995,12221.53
4,4822,5708.19
5,5651,5764.40
6,15344,12547.64
7,8492,8231.49
8,8565,7369.40
9,7185,6642.79



Top 15 features mais importantes (RF):


,feature,importance
1,Customers,0.666776
2,Promo,0.100912
4,CompetitionDistance,0.047990
0,Store,0.035103
9,DayOfWeek,0.031205
13,StoreType_d,0.020195
10,WeekOfYear,0.018129
11,StoreType_b,0.014925
8,Day,0.014372
15,Assortment_c,0.013156
